In [ ]:
"""
  微调Qwen-VL模型：本文采用finetune PEFT方法微调（参数高效微调）
  方法：1、通过json标注图片正确答案的数据集，或者通过labelwise、labelImg、Label Studio标注平台进行标注
       2、使用PEFT+LoRA方法微调模型
  注： 1、没有使用QLoRA方法进行量化后微调模型
      2、Qwen-VL微调框架地址：https://github.com/QwenLM/Qwen2.5-VL、https://github.com/QwenLM/Qwen2.5-VL/tree/main/qwen-vl-finetune

主要有两种主流的微调方法，适用于不同的硬件条件和需求。
1. 全参数微调（Full Fine-Tuning）
    描述：更新模型的所有参数。
    优点：通常能获得最好的性能，模型能充分学习新数据分布。
    缺点：极其消耗显存，需要大量的GPU资源（可能需要多卡A100 80G及以上），训练成本高。
    适用场景：数据量大、任务复杂、且拥有充足计算资源的情况。

2. 参数高效微调（Parameter-Efficient Fine-Tuning, PEFT）
这是目前最流行的微调方式，尤其是对于大模型。最常用的方法是 LoRA (Low-Rank Adaptation)。
    描述：冻结原模型的所有权重，仅向模型中插入少量的可训练“旁路”矩阵（Adapter）来模拟全参数更新。
    优点：
        显存需求大幅降低：通常只需全参数微调 1/3 甚至更少的显存。
        训练速度快：可训练参数少，计算量小。
        产出小：最终只需要保存和分发小小的 Adapter 权重（几MB到几百MB），而不是整个模型（几个GB）。
        可移植性强：一个基础模型可以搭配多个不同任务的Adapter。
    缺点：性能可能略低于全参数微调（但在大多数情况下足够好）。
    适用场景：绝大多数情况，特别是资源有限的个人开发者或实验室。
"""

In [ ]:
# 显示gradio和transformers的版本
import gradio
gradio.__version__
import transformers 
transformers.__version__
import torch
torch.__version__
import torch
from PIL import Image
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, AutoProcessor,
    TrainingArguments,
    Trainer,
    default_data_collator
)
from peft import LoraConfig, get_peft_model
import json
import os

In [ ]:
# 调用Qwen-VL本地的模型进行微调
model_name = "/root/autodl-tmp/models/Qwen/Qwen-VL-Chat"
# ==================== 配置部分 ====================
# LoRA 配置
lora_config = LoraConfig(
    r=64,  # LoRA的秩
    lora_alpha=16,  # 缩放参数
    target_modules=[  # 目标模块（针对Qwen-VL结构）
        "c_attn",  # 注意力层的QKV投影
        "attn.c_proj",  # 注意力输出投影
        "w1",  # MLP层
        "w2",
        "mlp.c_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# 训练参数
training_args = TrainingArguments(
    output_dir="/root/autodl-tmp/models/Qwen/qwen-vl-lora-finetuned",  # 输出目录
    per_device_train_batch_size=1,  # 根据GPU显存调整（A100可尝试2-4）
    gradient_accumulation_steps=8,  # 梯度累积步数
    learning_rate=2e-4,  # 学习率
    num_train_epochs=3,  # 训练轮数
    logging_dir="./logs",  # 日志目录
    logging_steps=10,  # 日志记录步数
    save_steps=100,  # 保存步数
    fp16=True,  # 使用混合精度训练（如果GPU支持）
    remove_unused_columns=False,  # 多模态训练必须为False，重要：不要自动移除未使用的列，否则会导致输入数据不一致
    dataloader_pin_memory=False,  # 避免内存复制问题
)

# 性能优化参数，它的意思是将序列长度填充到指定的倍数，这样可以提高模型的效率
def get_optimal_padding():
    """
    根据硬件选择最优的pad_to_multiple_of值
    """
    if torch.cuda.is_available():
        # 检查GPU架构
        gpu_name = torch.cuda.get_device_name(0)
        if "V100" in gpu_name or "A100" in gpu_name:
            return 64  # 新一代GPU
        else:
            return 8   # 一般GPU
    else:
        return 8       # CPU或其他设备

# 数据路径
dataset_path = "qwen-vl-dataset.json"  # 替换为你的数据集路径
image_folder = "./images"  # 图片所在文件夹路径

In [ ]:
# ==================== 数据预处理函数 ====================
def format_conversation(conversations):
    """将对话历史格式化为Qwen-VL接受的文本格式"""
    formatted_text = ""
    for turn in conversations:
        if turn["from"] == "user":
            formatted_text += f"<|im_start|>user\n{turn['value']}<|im_end|>\n"
        elif turn["from"] == "assistant":
            formatted_text += f"<|im_start|>assistant\n{turn['value']}<|im_end|>\n"
    # 添加assistant开始标记以引导生成
    formatted_text += "<|im_start|>assistant\n"
    return formatted_text

def collate_fn(batch):
    """处理批次数据的函数"""
    images = []
    texts = []
    
    for example in batch:
        try:
            # 1. 加载图片
            if example.get('image'):
                image_path = os.path.join(image_folder, example['image'])
                image = Image.open(image_path).convert('RGB')
                images.append(image)
            else:
                # 如果没有图片，使用空白图片或跳过
                images.append(Image.new('RGB', (224, 224), (255, 255, 255)))
            
            # 2. 格式化文本
            conversations = example['conversations']
            formatted_text = format_conversation(conversations)
            texts.append(formatted_text)
            
        except Exception as e:
            print(f"Error processing example {example.get('id', 'unknown')}: {e}")
            continue
    
    # 3. 使用processor处理图像和文本
    if images and texts:
        inputs = processor(
            images=images,
            text=texts,
            padding=True,
            return_tensors="pt",
            truncation=True,
            max_length=2048,  # 根据需求调整
            pad_to_multiple_of=get_optimal_padding(),  # 动态选择
        )
        
        # 4. 设置labels（用于计算loss）
        inputs["labels"] = inputs["input_ids"].clone()
        
        # 5. 将数据移动到GPU（如果可用）
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        return inputs
    else:
        return None

In [ ]:
# 1. 加载模型和处理器
print("Loading model and processor...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32, # 使用半精度节省显存
    device_map="auto",  # 多卡自动分配
    trust_remote_code=True # Qwen系列需要
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    # 方法1: 使用eos_token作为pad_token
    tokenizer.pad_token = tokenizer.eos_token
    # 方法2 (备用): 添加新的pad_token
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    print(f"Set pad_token to: {tokenizer.pad_token}")
processor = AutoProcessor.from_pretrained(
    model_name, 
    trust_remote_code=True,
    tokenizer=tokenizer  # 传入已配置好的tokenizer
)

# 检查配置是否正确
print(f"Final tokenizer pad_token: {tokenizer.pad_token}")

# 使用 tokenizer 处理纯文本
# text = "请描述这张图片的内容"
# encoded_text = tokenizer(text, return_tensors="pt")
# print(encoded_text.input_ids)  # 查看编码后的token IDs

# 2. 应用LoRA
print("Applying LoRA...")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
    
# 3. 加载数据集
print("Loading dataset...")
dataset = load_dataset('json', data_files=dataset_path)['train']
print(f"成功加载数据集，包含 {len(dataset)} 条数据")
    
# 4. 创建Trainer并开始训练
print("Starting training...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collate_fn,
    tokenizer=tokenizer,  # 直接使用配置好的tokenizer
)
    
# 开始训练
trainer.train()
    
# 5. 保存模型
print("Saving model...")
# 调用Trainer对象的save_model方法，将经过微调后的模型保存到指定的输出目录。
# 该方法会保存模型的权重参数以及配置文件，方便后续加载和使用。
trainer.save_model()

# 调用processor的save_pretrained方法，将处理器保存到训练参数中指定的输出目录。
# 处理器包含了分词器、图像预处理等相关配置，保存后可在推理时使用相同的处理流程。
processor.save_pretrained(training_args.output_dir)
    
print("Training completed!")

In [ ]:
"""微调后的推理示例"""
from peft import PeftModel
    
# 加载基础模型
base_model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True
)
    
# 加载LoRA权重
model = PeftModel.from_pretrained(base_model, training_args.output_dir)
    
# 合并权重（可选，合并后推理更快）
model = model.merge_and_unload()
    
#  加载处理器
processor = AutoProcessor.from_pretrained(
    training_args.output_dir, 
    trust_remote_code=True
)
    
# 准备输入
image_path = "test_image.jpg"  # 测试图片
question = "<image>\n请描述这张图片。"
    
image = Image.open(image_path).convert('RGB')
    
# 构建对话格式
messages = [
    {"from": "user", "value": question}
]
text = format_conversation(messages)
    
# 处理输入
inputs = processor(
    images=[image],
    text=text,
    return_tensors="pt",
).to(model.device)
    
# 生成回答（显式指定生成参数）
# 调用模型的generate方法生成文本，传入处理后的输入数据以及生成参数
# **inputs: 将处理后的输入数据以关键字参数的形式传入
# max_new_tokens=512: 指定模型最多生成512个新的token
# do_sample=True: 开启采样策略，使得生成结果更具多样性
# temperature=0.7: 控制采样时的随机性，值越小生成结果越确定，值越大随机性越强
# top_p=0.9: 使用核采样，只从概率累计和达到0.9的token中进行采样
generated_ids = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)
    
# 解码输出
generated_text = processor.batch_decode(
    generated_ids, 
    skip_special_tokens=True
)[0]
    
# 打印结果
print("Generated response:", generated_text)